Data Science 100 Project: Individual Project Planning Stage  | Owen Chen 73602740

(1) Data Description 

players.csv: This data set contains 7 variables and 196 observations.

#### Table 1. Variable Names, Types and Meaning in players.csv

| Variable     | Type      | Meaning                                                        |
|---------------|-----------|----------------------------------------------------------------|
| experience    | factor    |Minecraft Gaming Experience level (Veteran, Pro, Amateur, Regular).        |
| subscribe     | logical   | Status of player's subscription (TRUE/FALSE).             |
| hashedEmail   | character | Anonymous email addresses of players and session data.                             |
| played_hours  | double    | Total number of hours played by the individual on the server.                                  |
| name          | character |The Player's Display name                                             |
| gender        | factor    | Gender of the player                                            |
| Age           | integer   | Player's age in years.                                     |

sessions.csv: This data set contains 5 variables and 1535 observations

#### Table 2. Variable Names, Types and Meaning in sessions.csv

| Variable     | Type      | Meaning                                                        |
|---------------|-----------|----------------------------------------------------------------|
| hashedEmail   | Character    |Anonymous email addresses of players and session data.        |
| start_time          |Character |The timestamp start of a gameplay session in Milliseconds.
| end_time            |Character |The timestamp stop of a gameplay session in Milliseconds.                           |
| original_start_time |Double    |Start time in Epoch Milliseconds.                                      |
| original_end_time   |Double    |End time in Epoch Milliseconds.                                       |


For the purpose of this investigation, we will be using the players.csv dataset provided by DSCI 100.

(2)Questions:

One broad question that we have chosen is 
Can the following variables of gender, age, subscription, and experience be used to reliably predict the total hours played by the respective individual?

To answer this question, only the players.csv dataset will be used, since the question does not require any variables provided in the sessions.csv dataset. The required variables to answer the question would include experience, gender, age, subscription, and played_hours. Before using data, we need to wrangle the data to exclude the unnecessary variables within the players.csv.

(3)Exploratory Data Analysis and Visualization:

In [ ]:
    library(tidyverse)
    library(readr)
    library(dplyr)
    library(ggplot2)
    library(tidymodels)
    library(knitr)

In [ ]:
player_data<- read_csv('players.csv')
sessions_data<-read_csv('sessions.csv')

Firstly, we need to use as_factor for all categorical variables so that they can be grouped. Otherwise, no further wrangling is needed, as the player_csv dataset is already tidy, as each observation is a singular person, every variable has its own column, and each value is a single cell.

In [ ]:
mean_summary <- player_data |>
  summarize(
    Mean_Age = mean(Age, na.rm = TRUE),
    Mean_Played_Hours = mean(played_hours, na.rm = TRUE)
  ) |>
  pivot_longer(cols = everything(), names_to = "Variable", values_to = "Mean_Value") |>
  mutate(Mean_Value = round(Mean_Value, 2))


kable(mean_summary, caption = "3. Mean Values of Quantitative Variables in Players Dataset")

In [ ]:
distribution_total_hours_played <- ggplot(player_data, aes(x = played_hours)) +
  geom_histogram(binwidth = 5, fill = "steelblue", color = "white") +
  labs(
    title = "Distribution of Total Hours Played by Players",
    x = "Total Hours Played",
    y = "Number of Players"
  ) +
  theme_minimal(base_size = 16)
distribution_total_hours_played

In [ ]:
average_hours_played_experience <-player_data |>
  group_by(experience) |>
  summarize(mean_hours = mean(played_hours, na.rm = TRUE)) |>
  ggplot(aes(x = experience, y = mean_hours, fill = experience)) +
  geom_col(show.legend = FALSE) +
  labs(
    title = "Average Play Hours by Experience Level",
    x = "Experience Level",
    y = "Average Hours Played"
  ) +
  theme_minimal(base_size = 16)
average_hours_played_experience 


In [ ]:
Total_hours_played_age<- ggplot(player_data, aes(x = Age, y = played_hours)) +
  geom_point(alpha = 0.5, color = "darkgreen") +
  geom_smooth(method = "lm", se = FALSE, color = "blue") +
  labs(
    title = "Relationship Between Age and Total Hours Played",
    x = "Age (years)",
    y = "Total Hours Played"
  ) +
  theme_minimal(base_size = 16)
Total_hours_played_age


(4) Method and Plan

The method I will use to address the question is knn-regression. This method will help predict the total hours played using Age as the quantitative predictor variable, since the other variables are categorical. Thus I plan to split the data into multiple subsets and perform knn-regression on each individual subset. Since the variable experience has 5 categories, subscribe has 2 categories and gender has 7 categories, there would be 70 potential subsets of data (5 x 2 x 7). Then, k-nn regression will be performed with Age as the predictor and played_hours as the response variable, since categorical variables cannot be predictors for k-nn regression. 
This method is appropriate because it can capture nonlinear relationships between age and total playtime. Furthermore, due to the mix of categorical and quantitative data, splitting the data into subsets is most appropriate so that knn-regression can still be performed.

The only assumptions needed to apply this method is that players with similar ages within each category subset would also be expected to have similar playtime. Otherwise, no other assumptions are nessacary.

A limitation is that the method would not work as well with variable combinations that have small sample sizes, due to lack of neighbors to make reliable predictions. Furthermore, splitting data into subsets reduces the overall sample size, which further reduces the reliability of the predictions.

Before applying the model, the plan is to perform cross validation within each subset, which would help select the opitmal k value and model for each variable combination. However, this is only possible with variable combinations that have sufficient sample size. Prior to the cross validation, the data will be split 90% into training and 10% into testing. The larger split into training is to ensure that there is enough data for model fitting 